In [ ]:
import pandas as pd

# Convert Excel to CSV on the fly for debugging
excel_file = 'rawTransferSarnia.xlsx'  # your Excel input file
csv_file = 'rawTransferSarnia.csv'    # temp CSV file to be created

df = pd.read_excel(excel_file)
df.to_csv(csv_file, index=False)

In [1]:
import pandas as pd
import random
import math
import time
import datetime

start_time = time.time()
print("Module Started")

# Read the CSV file
df = pd.read_csv('rawTransferSarnia.csv')

# Create a dictionary to store the available to transfer values for each store
store_columns = { 'Chatham': 'Chatham AVA', 'Exmouth': 'Exmouth AVA', 'Lakeshore': 'Lakeshore AVA', 'Trudeau': 'Trudeau AVA'}

# Create a list to store the transfer operations
transfers = []

# Create a dictionary to store the updated available numbers at each store for each SKU
updated_available_numbers = {store: [] for store in store_columns}

# Percentage threshold for transfers (adjust as needed)
transfer_percentage = 0.25  # 50%

# Tolerance for determining when quantities are mostly equal (adjust as needed)
equality_tolerance = 3


# Maximum number of iterations (adjust as needed)
inv_columns = {
    'Chatham': 'Chatham INV',
    'Exmouth': 'Exmouth INV',
    'Lakeshore': 'Lakeshore INV',
    'Trudeau': 'Trudeau INV'
}

# Store the initial inventory values
initial_inventory = df[['Chatham INV', 'Exmouth INV', 'Lakeshore INV', 'Trudeau INV']].copy()
for index in df.index:
    sku = df.loc[index, 'SKU']
    product_name = df.loc[index, 'Product Name']
    category = df.loc[index, 'Category']
    transfer_possible = True
    out_of_stock = True
    max_iterations = 0
    while transfer_possible and max_iterations < 100:
        max_iterations += 1
        donor_store = max(store_columns, key=lambda s: df.loc[index, store_columns[s]])
        receiver_store = min(store_columns, key=lambda s: df.loc[index, store_columns[s]])
        donor_max_quantity = df.loc[index, store_columns[donor_store]]
        receiver_min_quantity = df.loc[index, store_columns[receiver_store]]
        while out_of_stock:
            oos_donor_store = max(inv_columns, key=lambda s: df.loc[index, inv_columns[s]])
            oos_donor_avail_quantity = df.loc[index, inv_columns[oos_donor_store]]
            sorted_stores_OS = sorted(inv_columns, key=lambda store: df.loc[index, inv_columns[store]], reverse=False)
            from_store_OS = oos_donor_store
            to_store_OS = sorted_stores_OS[0]
            if (oos_donor_avail_quantity > 1 and df.loc[index, inv_columns[to_store_OS]] == 0):
                out_of_stock = True
            else:
                out_of_stock = False
            if (oos_donor_avail_quantity > 1 and out_of_stock):
                amount_to_transfer = 1        
                df.loc[index, inv_columns[from_store_OS]] -= amount_to_transfer
                df.loc[index, inv_columns[to_store_OS]] += amount_to_transfer
                df.loc[index, store_columns[from_store_OS]] -= amount_to_transfer
                transfers.append({
                    'SKU': df.loc[index, 'SKU'],
                    'Product Name': df.loc[index, 'Product Name'],
                    'Category': df.loc[index, 'Category'],
                    'From Store': from_store_OS,
                    'To Store': to_store_OS,
                    'Amount': amount_to_transfer
                })
        donor_store = max(store_columns, key=lambda s: df.loc[index, store_columns[s]])
        receiver_store = min(store_columns, key=lambda s: df.loc[index, store_columns[s]])
        donor_max_quantity = df.loc[index, store_columns[donor_store]]
        receiver_min_quantity = df.loc[index, store_columns[receiver_store]]
        if (receiver_min_quantity >= 0):
            if (donor_max_quantity - receiver_min_quantity > 1) and (receiver_min_quantity < 7):
        # Sort the stores by available to transfer in descending order
                sorted_stores = sorted(store_columns, key=lambda store: (df.loc[index, store_columns[store]], store), reverse=True)

            # Calculate the average available to transfer for each store
                avg_transfer = round(
                    sum(df.loc[index, store_columns[store]] for store in store_columns) / len(store_columns)
                )
                    
            # Transfer products from the store with the most quantity to the store with the least inventory
                from_store = sorted_stores[0]
                to_store = min(sorted_stores[1:], key=lambda store: df.loc[index, store_columns[store]])

            # If there's a tie for the store with the most quantity, randomize between Wyandotte and Huron before Tecumseh
                if df.loc[index, store_columns[from_store]] == df.loc[index, store_columns[sorted_stores[1]]]:
                    if 'Lakeshore' not in sorted_stores:
                        to_store = random.choice(['Exmouth', 'Trudeau'])

            # Calculate the whole number amount to transfer (using a percentage)
                amount_to_transfer = 1

                # Update the available to transfer values for both stores
                df.loc[index, store_columns[from_store]] -= amount_to_transfer
                df.loc[index, store_columns[to_store]] += amount_to_transfer
                updated_from_value = df.loc[index, store_columns[from_store]]
                updated_to_value = df.loc[index, store_columns[to_store]]
                transfers.append({
                    'SKU': df.loc[index, 'SKU'],
                    'Product Name': df.loc[index, 'Product Name'],
                    'Category': df.loc[index, 'Category'],
                    'From Store': from_store,
                    'To Store': to_store,
                    'Amount': amount_to_transfer
                })
                
            else:
                transfer_possible = False
        else:
            if  donor_max_quantity > 1 and receiver_min_quantity < 6 :
                sorted_stores = sorted(store_columns, key=lambda store: (df.loc[index, store_columns[store]], store), reverse=True)
                avg_transfer = round(
                    sum(df.loc[index, store_columns[store]] for store in store_columns) / len(store_columns)
                )
                from_store = sorted_stores[0]
                to_store = min(sorted_stores[1:], key=lambda store: df.loc[index, store_columns[store]])
                if df.loc[index, store_columns[from_store]] == df.loc[index, store_columns[sorted_stores[1]]]:
                    if 'Lakeshore' not in sorted_stores:
                        to_store = random.choice(['Exmouth', 'Trudeau'])
                amount_to_transfer = 1
                df.loc[index, store_columns[from_store]] -= amount_to_transfer
                df.loc[index, store_columns[to_store]] += amount_to_transfer
                updated_from_value = df.loc[index, store_columns[from_store]]
                updated_to_value = df.loc[index, store_columns[to_store]]
                transfers.append({
                    'SKU': df.loc[index, 'SKU'],
                    'Product Name': df.loc[index, 'Product Name'],
                    'Category': df.loc[index, 'Category'],
                    'From Store': from_store,
                    'To Store': to_store,
                    'Amount': amount_to_transfer
                })
            else:
                transfer_possible = False    
                
# Create a new DataFrame from the transfers list
transfers_df = pd.DataFrame(transfers, columns=['SKU', 'Product Name', 'Category', 'From Store', 'To Store', 'Amount'])

# Save the result to a new CSV file
transfers_df.to_csv('transfers_output_sarnia.csv', index=False)

# Store the updated available numbers at each store for each SKU
for _, row in df.iterrows():
    for store in store_columns:
        updated_available_numbers[store].append(row[store_columns[store]])

# Create a new DataFrame from the updated available numbers
updated_available_df = pd.DataFrame(updated_available_numbers, columns=['Chatham AVA', 'Exmouth AVA', 'Lakeshore AVA', 'Trudeau AVA'])

# Concatenate the SKU, Product Name, Category, and initial inventory columns with the updated available numbers
updated_available_df = pd.concat([df[['SKU', 'Product Name', 'Category', 'Chatham INV', 'Exmouth INV', 'Lakeshore INV', 'Trudeau INV']], updated_available_df], axis=1)

# Save the updated available numbers to a new CSV file
updated_available_df.to_csv('updated_available_numbers_sarnia.csv', index=False)

end_time = time.time()
print(f"Total execution time: {end_time - start_time:.2f} seconds")


Module Started
Total execution time: 7.30 seconds


In [2]:
import csv

def condense_csv(input_file, output_file):
    data = {}

    # Read CSV file and store data in a dictionary
    with open(input_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  # Assuming the first row contains headers
        for row in reader:
            key = tuple(row[i] for i in [0, 3, 4])  # Columns 1, 4, and 5
            if key in data:
                data[key][5] = str(int(data[key][5]) + int(row[5]))  # Combine values in column 6
            else:
                data[key] = row

    # Write condensed data to a new CSV file
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)
        for row in data.values():
            writer.writerow(row)

# Example usage
input_file = 'transfers_output_sarnia.csv'  # Replace with your CSV file path
output_file = 'transfers_output_good_Sarnia.csv'  # Replace with the desired output file path
condense_csv(input_file, output_file)
